## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns and other columns non-detrimental to the success of the organization, 'EIN' and 'NAME'.
df_application = application_df.drop(columns=['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS'])
df_application.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [3]:
# Determine the number of unique values in each column.
df_application.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_value_counts = df_application['APPLICATION_TYPE'].value_counts()
application_type_value_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# Decrease the number of bins for optimization
application_types_to_replace = application_type_value_counts[application_type_value_counts<725].index

# Replace in dataframe
for app in application_types_to_replace:
    df_application['APPLICATION_TYPE'] = df_application['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
df_application['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_value_counts = df_application['CLASSIFICATION'].value_counts()
classification_value_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_value_counts_gt1 = classification_value_counts[classification_value_counts > 1]
classification_value_counts_gt1

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_value_counts[classification_value_counts<1883].index


# Replace in dataframe
for cls in classifications_to_replace:
    df_application['CLASSIFICATION'] = df_application['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
df_application['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [9]:
# Look at ASK_AMT value counts
ask_amt_value_counts = df_application['ASK_AMT'].value_counts()
ask_amt_value_counts


ASK_AMT
5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: count, Length: 8747, dtype: int64

In [20]:

# Creating custom bins for ASK_AMTs
# Decrease the number of bins for optimization.

# Previous Code:
# bins = [0, 4999, 10000, 14999, 20000, 24999, 100000, 499999, 1000000, 19999999, 30000000, 40000000]
# bin_labels = ['less than 5,000', '5,000-9,999', '10,000-14,999', '15,000-19,999', '20,000-24,999', '25,000-99,999', '100,000-499,999', '500,000-999,999', '1,000,000-19,999,999', '20,000,000-29,999,999', '30,000,000-39,999,999']

# New Code:
bins = [0, 9999, 15000, 24999, 100000, 499999, 1000000, 40000000]
bin_labels = ['5,000 - 9,999', '10,000-14,999', '15,000-24,999', '25,000-99,999',
              '100,000-499,999', '500,000-999,999', '1,000,000+']

# Ensure the number of labels matches the number of bins minus one
assert len(bins) - 1 == len(bin_labels), "Number of bin labels must be one less than the number of bin edges."

# Create bins using cut() with labels
df_application['ASK_AMT_BINS'] = pd.cut(df_application['ASK_AMT'], bins=bins, labels=bin_labels)

# Display the frequency of values in each bin
print(df_application['ASK_AMT_BINS'].value_counts())


ASK_AMT_BINS
5,000 - 9,999      25946
25,000-99,999       2687
100,000-499,999     2303
1,000,000+          1395
15,000-24,999        747
500,000-999,999      651
10,000-14,999        388
Name: count, dtype: int64


In [22]:
# Convert categorical data to numeric with `pd.get_dummies`
needs_encoding = ['APPLICATION_TYPE', 'AFFILIATION',	'CLASSIFICATION',	'USE_CASE',	'ORGANIZATION',	'INCOME_AMT',	'ASK_AMT_BINS']
df_encoded_applications = pd.get_dummies(df_application, columns=needs_encoding)
df_encoded_applications = df_encoded_applications.astype(int)
df_encoded_applications.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,"ASK_AMT_BINS_5,000 - 9,999","ASK_AMT_BINS_10,000-14,999","ASK_AMT_BINS_15,000-24,999","ASK_AMT_BINS_25,000-99,999","ASK_AMT_BINS_100,000-499,999","ASK_AMT_BINS_500,000-999,999","ASK_AMT_BINS_1,000,000+"
0,5000,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,108590,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,5000,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,6692,1,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [23]:
# Split our preprocessed data into our features and target arrays
X = df_encoded_applications.copy()
X.drop(['IS_SUCCESSFUL'], axis=1)

y = df_encoded_applications['IS_SUCCESSFUL'].values.reshape(-1, 1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [24]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
print(f'shape of training data: {X_train_scaled.shape}')
X_test_scaled = X_scaler.transform(X_test)

shape of training data: (25724, 47)


## Compile, Train and Evaluate the Model

In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

# Add one more hidden layer for optimization

deep_neural_net = df_encoded_applications.shape[1]

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=deep_neural_net))

# Add another hidden layer
nn_model.add(tf.keras.layers.Dense(units=60, activation='relu'))

# Second hidden layer.  Now the third hidden layer
nn_model.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 80)                3840      
                                                                 
 dense_5 (Dense)             (None, 60)                4860      
                                                                 
 dense_6 (Dense)             (None, 30)                1830      
                                                                 
 dense_7 (Dense)             (None, 1)                 31        
                                                                 
Total params: 10561 (41.25 KB)
Trainable params: 10561 (41.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [28]:
# Train the model
# Add a callback function that saves the model at a frequency of 5 epochs
tf.keras.backend.clear_session()
weight_checkpoint = 'model_weights.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=weight_checkpoint,
    save_weights_only=True,
    save_freq=5
)
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[model_checkpoint_callback])

nn_model.load_weights(weight_checkpoint)

Epoch 1/100
804/804 [==============================] - 5s 5ms/step - loss: 0.0418 - accuracy: 0.9883 - val_loss: 0.0421 - val_accuracy: 0.9957
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0058 - accuracy: 0.9994 - val_loss: 3.1228e-04 - val_accuracy: 0.9999
Epoch 3/100
804/804 [==============================] - 5s 6ms/step - loss: 0.0025 - accuracy: 0.9997 - val_loss: 0.0021 - val_accuracy: 0.9995
Epoch 4/100
804/804 [==============================] - 5s 6ms/step - loss: 0.0024 - accuracy: 0.9995 - val_loss: 0.0029 - val_accuracy: 0.9997
Epoch 5/100
804/804 [==============================] - 6s 7ms/step - loss: 5.1983e-04 - accuracy: 0.9999 - val_loss: 0.0017 - val_accuracy: 0.9997
Epoch 6/100
804/804 [==============================] - 4s 5ms/step - loss: 5.3822e-04 - accuracy: 0.9998 - val_loss: 8.9878e-04 - val_accuracy: 0.9998
Epoch 7/100
804/804 [==============================] - 4s 4ms/step - loss: 5.7267e-04 - accuracy: 0.9998 - val_loss: 0.0035 - 

In [31]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print("Loss: {:.10f}, Accuracy: {:.4f}".format(model_loss, model_accuracy))

268/268 - 2s - loss: 1.1950e-05 - accuracy: 1.0000 - 2s/epoch - 6ms/step
Loss: 0.0000119496, Accuracy: 1.0000


In [32]:
# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity_Optimization.h5")

# Download the model to computer
from google.colab import files
files.download('AlphabetSoupCharity_Optimization.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>